In [ ]:
import os
import pickle
from functools import partial

import numpy as np

from ad_afqmc import config
from ad_afqmc import pyscf_interface


import numpy as np
#import scipy
from pyscf import __config__, ao2mo, mcscf, scf

In [ ]:
def run_afqmc(options=None, script=None, mpi_prefix=None, nproc=None):
    if options is None:
        options = {}
    with open("options.bin", "wb") as f:
        pickle.dump(options, f)
    if script is None:
        path = os.path.abspath(__file__)
        dir_path = os.path.dirname(path)
        script = f"{dir_path}/mpi_jax.py"
    use_gpu = config.afqmc_config["use_gpu"]
    gpu_flag = "--use_gpu" if use_gpu else ""
    if mpi_prefix is None:
        if use_gpu:
            mpi_prefix = ""
        else:
            mpi_prefix = "mpirun "
    if nproc is not None:
        mpi_prefix += f"-np {nproc} "
    os.system(
        f"export OMP_NUM_THREADS=1; export MKL_NUM_THREADS=1; {mpi_prefix} python {script} {gpu_flag}"
    )
    try:
        ene_err = np.loadtxt("ene_err.txt")
    except:
        print("AFQMC did not execute correctly.")
        ene_err = 0.0, 0.0
    return ene_err[0], ene_err[1]